In [4]:
import networkx as nx
# import matplotlib.pyplot as plt
from typing import Type
# from simulator import CovertGenerator, DarkGenerator
import numpy as np
import pandas as pd
import json
from tqdm import trange
from utility import read_gml, getRobustness, HXA

## Heuristic based

In [ ]:
for i in trange(100):
    hxa_dct, reward_dct = {}, {}
    g = read_gml(data_dir="./ba/", file_name=f"g_{i}")
    G = g.copy()
    for method in ['HDA', 'HBA', 'HCA', 'HPRA']:
        sol, reward = HXA(g=g, method=method)
        hxa_dct[method] = sol
        reward_dct[method] = (np.cumsum(reward) / g.number_of_nodes()).tolist()
        
    with open(f"./ba/hxa_node_hist/g_{i}.json", "w") as json_file:
        json.dump(hxa_dct, json_file) 
    with open(f"./ba/hxa_reward_hist/g_{i}.json", "w") as json_file:
        json.dump(reward_dct, json_file)

## FINDER 

In [ ]:
for i in trange(100):
    finder_dct = dict()
    reward_lst = []
    g = read_gml(data_dir="./ba/", file_name=f"g_{i}")
    G = g.copy()
    df = pd.read_csv(f"./ba/finder_node_hist/g_{i}.txt", header=None).rename(columns={0: "round", 1: "node"})
    
    for row_idx, (round, node) in df.iterrows():
        reward_lst.append(getRobustness(g, G, int(node)))
        
    for _ in range(g.number_of_nodes() - len(reward_lst)):
        GCCsize = len(max(nx.connected_components(g), key=len))
        reward_lst.append(1 - 1/GCCsize)

    reward_lst = (np.cumsum(reward_lst) / g.number_of_nodes()).tolist()

    with open(f"./ba/finder_reward_hist/g_{i}.json", "w") as json_file:
        json.dump(reward_lst, json_file)


## FINDER action w.r.t HXA

In [ ]:
from scipy.stats import percentileofscore

rank_dct = {
    "HDA": [], "HBA": [], 
    "HCA": [], "HPRA": []
}

for i in range(100):
    G = read_gml(data_dir="./ba/", file_name=f"g_{i}")
    df = pd.read_csv(f"./ba/finder_node_hist/g_{i}.txt", header=None).rename(columns={0: "round", 1: "node"})    
    
    rank_lst = []
    for _, (round, n) in df.iterrows():
        try:
            for method in ['HDA', 'HBA', 'HCA', 'HPRA']:
                if method == 'HDA':
                    dc = nx.degree_centrality(G)
                elif method == 'HBA':
                    dc = nx.betweenness_centrality(G)
                elif method == 'HCA':
                    dc = nx.closeness_centrality(G)
                elif method == 'HPRA':
                    dc = nx.pagerank(G)

                rank = percentileofscore(list(dc.values()), dc[n])
                rank_lst.append(rank)

            removed = [n for n in G.neighbors(int(n))] + [int(n)]
            for n in removed:
                G.remove_node(n)

        except:
            pass
    rank_dct[method].append(rank_lst)

In [ ]:
for method in ['HDA', 'HBA', 'HCA', 'HPRA']:
    avg_lst = []
    for rank_lst in rank_dct[method]:
        for i in rank_lst:
            avg_lst.append(i)
#         avg_lst.append(np.mean(rank_lst))
    plt.xlabel('Pertile')
    plt.ylabel('Frequency')
    
    plt.hist(avg_lst, bins=20, color="blue")
    plt.title(f"BA: {method} percetntile (all_round)")
    plt.savefig(f"./ba/{method}_all_round.png") 
    